# 과정
1. 교차로 dataset 를 yolo txt 형식에 맞게 변환 
(Zebra_cross : 0,G_Signal : 1,R_Signal : 2 ) 
    => 왜 ? 이것도 나중에 개별 데이터셋으로 활용하기 위해 (1회성 데이터셋이 아닌 나중에도 써먹기 위해)

2. 변환된 파일을 zip파일로 저장

3. 변환된 파일과 버스 데이터를 합치는데   
버스 데이터와 교차로 데이터의 인덱스를 맞춰야됨

4. 그냥.. 원본 데이터셋에서 변형된 버전을 하나 만들기로함...

5. 그러고 버스랑 합친 MergedDataset 파일 만들고 그거 zip 묶어서 저장~

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

73616개

# Cross 데이터셋 변형 1 (0, 1, 2)

In [ ]:
import torch
import os
from os import walk, getcwd
import shutil
import random
import numpy as np

from sklearn.model_selection import train_test_split

from tqdm import tqdm

from glob import glob

import yaml
import json

from IPython.display import Image

from pathlib import Path
import cv2

In [ ]:

## Unzip Image File
%cd /content
!unzip -qq "/content/gdrive/MyDrive/Colab Notebooks/YOLO V5/cross.zip" ## Folder 6개가 생김
print('Complete!')

In [ ]:
'''
압축이 해제된 폴더들은 Object의 수에 따라 1~6으로 구분 해둔 것임
따라서 일단 안의 파일들을 다 하나로 합쳐준 다음 Train Val 분리를 해주기로 결정
'''

os.makedirs('./dataset') ## 1~6으로 분할 되어있는 파일을 다 모아줄 폴더 생성

to_here = '/content/dataset'


for i in tqdm(range(1,7)) :
    get_file = os.listdir(f'/content/교차로정보 데이터셋_bbox_{i}') ## 폴더 내의 파일들 리스트로 묶음
    for j in get_file :
        shutil.move(f'/content/교차로정보 데이터셋_bbox_{i}/'+j, to_here) ## dataset 폴더로 모두 이동
    os.rmdir(f'/content/교차로정보 데이터셋_bbox_{i}') 

In [ ]:
## json Label Data를 이제 txt로 바꿔줘야됨 (이게 제일 빡... ㅠ)
## class | x_center | y_center | width | height 순서임

## Train Data 변환 ##
mypath = "/content/dataset/"

os.makedirs('/content/yolo_texts') 
outputpath = '/content/yolo_texts/'




def convert(size, box):
    dw = 1. / (size[0])
    dh = 1. / (size[1])
    x = (box[0] + box[1]) / 2.0 - 1
    y = (box[2] + box[3]) / 2.0 - 1
    w = box[1] - box[0]
    h = box[3] - box[2]
    x = x * dw
    w = w * dw
    y = y * dh
    h = h * dh
    return (x, y, w, h)

json_name_list = []
for file in os.listdir(mypath):
    if file.endswith(".json"):
        json_name_list.append(file)

## Class 정의
obj_classes = ['Zebra_Cross','G_Signal','R_Signal']


for json_file in tqdm(json_name_list) : 

    ## txt 파일로 바꿔줄 이름 !
    txt_file_path = outputpath+json_file.split('.')[0] + '.txt'
    txt_label = open(txt_file_path, 'w') ## 이 파일에다가 쓸 거야!

    json_obj = json.load(open(mypath+json_file, 'r', encoding='utf-8'))  ## json 파일 오픈
    height = json_obj['imageHeight'] ## image에서 i 대문자 아니고 소문자임 ㅋㅋㅋ ㅠㅠㅠㅠㅠ
    width = json_obj['imageWidth'] ## 이미지의 높이 정보, 너비정보
    for idx, i in enumerate(json_obj['shapes']) : 
        label = i['label']
        if label == '1' : ## 중간에 라벨에 1이라고 잘못한 게 있는듯 그거 패스해버림
            continue
        points = i['points']
        if (i["shape_type"] == 'rectangle'):
            points = np.array(i["points"])
            xmin = min(points[:, 0]) if min(points[:, 0]) > 0 else 0
            xmax = max(points[:, 0]) if max(points[:, 0]) > 0 else 0
            ymin = min(points[:, 1]) if min(points[:, 1]) > 0 else 0
            ymax = max(points[:, 1]) if max(points[:, 1]) > 0 else 0

            ## 이상한 거 없나 체크
            if xmax <= xmin:
                pass
            elif ymax <= ymin:
                pass
            else:
                ## 이상 없으면 변환시킴
                bbox_labelme_float   = (float(xmin), float(xmax), float(ymin), float(ymax))
                bbox_yolo_normalized = convert((width, height), bbox_labelme_float)

                ## class_id는 obj_classes에 맞는 index가 됨
                ## 횡단보도 = 0 , 초록불 = 1 , 빨간불 =2            
                class_id = obj_classes.index(label)

                ## 드디어~~ txt 파일로 작성!!!!! (한줄)
                txt_label.write(str(class_id)+ ' ' + ' '.join([str(a) for a in bbox_yolo_normalized])+'\n')

    ## 다 썼으면 닫기
    txt_label.close()

In [ ]:
img_lst = []

get_file = os.listdir(f'/content/dataset') ## 폴더 내의 파일들 리스트로 묶음
for j in get_file :
    if j.endswith('.png') : 
        img_lst.append(j)
    if j.endswith('.jpg') : 
        img_lst.append(j)

In [ ]:
import shutil 
import tqdm

for img in tqdm(img_lst) : 
    shutil.move(f'/content/dataset/{img}', '/content/yolo_texts')

In [ ]:
len(glob('/content/yolo_texts/*'))

YOLO Texts로 변환시킨 데이터셋만 zip일단 시킴(안전빵)

In [ ]:
!zip -r /content/yolo_texts.zip /content/yolo_texts

In [ ]:
print('이미지 수 \n',
      len(glob('/content/yolo_texts/*.png')) + len(list(glob('/content/yolo_texts/*.jpg'))))
print('txt로 변환한 라벨 수 \n',
      len(glob('/content/yolo_texts/*.txt')))

print('총 파일 수 : {이전과 일치, 모든 파일 정상 변환 완료}\n',
      len(glob('/content/yolo_texts/*')))


일단 다운로드

In [ ]:
from google.colab import files
files.download('/content/yolo_texts.zip')

# Cross 변형 2 (12, 13, 14)

In [ ]:
## Unzip Image File
%cd /content
!unzip -qq "/content/gdrive/MyDrive/Colab Notebooks/YOLO_V5/cross.zip" ## Folder 6개가 생김
print('Complete!')

In [ ]:
import os
from tqdm import tqdm
from glob import glob
import shutil




os.makedirs('./dataset') ## 1~6으로 분할 되어있는 파일을 다 모아줄 폴더 생성

to_here = '/content/dataset'


for i in tqdm(range(1,7)) :
    get_file = os.listdir(f'/content/교차로정보 데이터셋_bbox_{i}') ## 폴더 내의 파일들 리스트로 묶음
    for j in get_file :
        shutil.move(f'/content/교차로정보 데이터셋_bbox_{i}/'+j, to_here) ## dataset 폴더로 모두 이동
    os.rmdir(f'/content/교차로정보 데이터셋_bbox_{i}') 

In [ ]:
import json
import numpy as np

mypath = "/content/dataset/"

os.makedirs('/content/ProjectVerCross') 
outputpath = '/content/ProjectVerCross/'




def convert(size, box):
    dw = 1. / (size[0])
    dh = 1. / (size[1])
    x = (box[0] + box[1]) / 2.0 - 1
    y = (box[2] + box[3]) / 2.0 - 1
    w = box[1] - box[0]
    h = box[3] - box[2]
    x = x * dw
    w = w * dw
    y = y * dh
    h = h * dh
    return (x, y, w, h)

json_name_list = []
for file in os.listdir(mypath):
    if file.endswith(".json"):
        json_name_list.append(file)

## Class 정의
obj_classes = ['Zebra_Cross','G_Signal','R_Signal']


for json_file in tqdm(json_name_list) : 

    ## txt 파일로 바꿔줄 이름 !
    txt_file_path = outputpath+json_file.split('.')[0] + '.txt'
    txt_label = open(txt_file_path, 'w') ## 이 파일에다가 쓸 거야!

    json_obj = json.load(open(mypath+json_file, 'r', encoding='utf-8'))  ## json 파일 오픈
    height = json_obj['imageHeight'] ## image에서 i 대문자 아니고 소문자임 ㅋㅋㅋ ㅠㅠㅠㅠㅠ
    width = json_obj['imageWidth'] ## 이미지의 높이 정보, 너비정보
    for idx, i in enumerate(json_obj['shapes']) : 
        label = i['label']
        if label == '1' : ## 중간에 라벨에 1이라고 잘못한 게 있는듯 그거 패스해버림
            continue
        points = i['points']
        if (i["shape_type"] == 'rectangle'):
            points = np.array(i["points"])
            xmin = min(points[:, 0]) if min(points[:, 0]) > 0 else 0
            xmax = max(points[:, 0]) if max(points[:, 0]) > 0 else 0
            ymin = min(points[:, 1]) if min(points[:, 1]) > 0 else 0
            ymax = max(points[:, 1]) if max(points[:, 1]) > 0 else 0

            ## 이상한 거 없나 체크
            if xmax <= xmin:
                pass
            elif ymax <= ymin:
                pass
            else:
                ## 이상 없으면 변환시킴
                bbox_labelme_float   = (float(xmin), float(xmax), float(ymin), float(ymax))
                bbox_yolo_normalized = convert((width, height), bbox_labelme_float)

                '''
                이 부분을 수정함.
                기존의 횡단보도, 신호등 이미지들을 12 13 14 으로
                '''

                if obj_classes.index(label) == 0 :
                    class_id = 12 ## Zebra_Cross : 12
                elif obj_classes.index(label) == 1 :
                    class_id = 13 ## G_Signal : 13
                elif obj_classes.index(label) == 2 :
                    class_id = 14 ## R_Signal : 14

                ## 드디어~~ txt 파일로 작성!!!!! (한줄)
                txt_label.write(str(class_id)+ ' ' + ' '.join([str(a) for a in bbox_yolo_normalized])+'\n')

    ## 다 썼으면 닫기
    txt_label.close()

In [ ]:
## 하나만 꺼내봤더니 잘 됨 ##

from glob import glob

ttxt = glob('/content/ProjectVerCross/*.txt')[0]

print(ttxt)
with open(ttxt, 'r') as f :
    lines = f.readlines()

print(lines)

In [ ]:
img_lst = []

get_file = os.listdir(f'/content/dataset') ## 폴더 내의 파일들 리스트로 묶음
for j in get_file :
    if j.endswith('.png') : 
        img_lst.append(j)
    if j.endswith('.jpg') : 
        img_lst.append(j)

In [ ]:
import shutil 
from tqdm import tqdm

for img in tqdm(img_lst) : 
    shutil.move(f'/content/dataset/{img}', '/content/ProjectVerCross')

In [ ]:
print('이미지 수 \n',
      len(glob('/content/ProjectVerCross/*.png')) + len(list(glob('/content/ProjectVerCross/*.jpg'))))
print('txt로 변환한 라벨 수 \n',
      len(glob('/content/ProjectVerCross/*.txt')))

print('총 파일 수 : {이전과 일치, 모든 파일 정상 변환 완료}\n',
      len(glob('/content/ProjectVerCross/*')))


# 여기서부터 합치기

In [ ]:
# %cd /content
# !unzip -qq "/content/gdrive/MyDrive/data/yolo_texts.zip" 

In [ ]:
%cd /content
!unzip -qq "/content/gdrive/MyDrive/Colab Notebooks/YOLO_V5/AugedBus.zip" 

In [ ]:
import os
from tqdm import tqdm



os.mkdir('/content/MergedDataset')


os.mkdir('/content/MergedDataset/train')
os.mkdir('/content/MergedDataset/train/images')
os.mkdir('/content/MergedDataset/train/labels')

os.mkdir('/content/MergedDataset/valid')
os.mkdir('/content/MergedDataset/valid/images')
os.mkdir('/content/MergedDataset/valid/labels')

In [ ]:
from sklearn.model_selection import train_test_split

img_lst = []
label_lst = []


get_file = os.listdir(f'/content/ProjectVerCross') ## 폴더 내의 파일들 리스트로 묶음
for j in get_file :
    if j.endswith('.png') : 
        img_lst.append(j)
    elif j.endswith('.jpg') : 
        img_lst.append(j)
    elif j.endswith('.txt') :
        label_lst.append(j)

    elif j.endwith('.json') : 
        continue

train_img , val_img = train_test_split(img_lst, test_size=0.1, random_state=42) ## train set과 validation set 분리

train_lab = []
val_lab = []

for i in tqdm(label_lst) :
    if i.split('.')[0] + '.jpg' in train_img :
        train_lab.append(i)
    elif i.split('.')[0] + '.png' in train_img :
        train_lab.append(i)

    elif i.split('.')[0] + '.jpg' in val_img :
        val_lab.append(i)

    elif i.split('.')[0] + '.png' in val_img :
        val_lab.append(i)

In [ ]:
print(f'Cross Train Image 수 : {len(train_img)}')
print(f'Cross Train Label 수 : {len(train_lab)}')
print(f'Cross Valid Image 수 : {len(val_img)}')
print(f'Cross Valid Label 수 : {len(val_lab)}')

In [ ]:
import shutil

for a, b in zip(train_img, train_lab) :
    shutil.move(f'/content/ProjectVerCross/{a}', '/content/MergedDataset/train/images')
    shutil.move(f'/content/ProjectVerCross/{b}', '/content/MergedDataset/train/labels')

for c,d in zip(val_img, val_lab) : 
    shutil.move(f'/content/ProjectVerCross/{c}', '/content/MergedDataset/valid/images')
    shutil.move(f'/content/ProjectVerCross/{d}', '/content/MergedDataset/valid/labels')

### 1차 확인

In [ ]:
train_path = list(sorted(glob('/content/MergedDataset/train/images/*')))
valid_path = list(sorted(glob('/content/MergedDataset/valid/images/*')))

train_label_path = list(sorted(glob('/content/MergedDataset/train/labels/*')))
valid_label_path = list(sorted(glob('/content/MergedDataset/valid/labels/*')))

print(train_path[:10])
print(train_label_path[:10])
print('++++++++++')
print(valid_path[:10])
print(valid_label_path[:10])

버스 옮겨줌

In [ ]:
tr_img_path = list(sorted(glob('/content/train/images/*')))
tr_lab_path = list(sorted(glob('/content/train/labels/*')))

vl_img_path = list(sorted(glob('/content/valid/images/*')))
vl_lab_path = list(sorted(glob('/content/valid/labels/*')))


for tr_img, tr_lab in zip(tr_img_path, tr_lab_path ) :
    shutil.move(tr_img, '/content/MergedDataset/train/images')
    shutil.move(tr_lab, '/content/MergedDataset/train/labels')    

for vl_img, vl_lab in zip(vl_img_path, vl_lab_path) :
    shutil.move(vl_img, '/content/MergedDataset/valid/images')
    shutil.move(vl_lab, '/content/MergedDataset/valid/labels')    

### 2차 확인
잘 됨... 뿌듯..

In [ ]:
train_path = list(sorted(glob('/content/MergedDataset/train/images/*')))
valid_path = list(sorted(glob('/content/MergedDataset/valid/images/*')))

train_label_path = list(sorted(glob('/content/MergedDataset/train/labels/*')))
valid_label_path = list(sorted(glob('/content/MergedDataset/valid/labels/*')))

print('Train 이미지 수 :',len(train_path))
print('Train 라벨 수 :',len(train_label_path))
print(train_path[:10])
print(train_label_path[:10])
print('++++++++++')
print('Valid 이미지 수 :',len(valid_path))
print('Valid 라벨 수 :',len(valid_label_path))
print(valid_path[:10])
print(valid_label_path[:10])



### 경로 알리미 파일 만들어주기

In [ ]:
with open('/content/MergedDataset/train.txt','w') as f :
    f.write('\n'.join(train_path) + '\n')

with open('/content/MergedDataset/valid.txt','w') as f :
    f.write('\n'.join(valid_path) + '\n')

# yaml 파일 수정하기

아마 한 에폭에 속도가 20~30분이 안 걸린다면
pathfile 경로를 txt가 아니라 폴더로 지정해서 그럼

In [ ]:
import yaml

In [ ]:
'''
train: ../train/images
val: ../valid/images

nc: 11
names: ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'bus']


이렇게 되어있는 걸

train: {YourTrainPathFile}
val: {YourValidPathFile}

nc: 14 
names: ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'bus', 'Zebra_Cross','G_Signal','R_Signal']

이렇게 바꿀 거임

'''

YourTrainPathFile = '/content/MergedDataset/train/images'
YourValidPathFile = '/content/MergedDataset/valid/images'

with open('/content/MergedDataset/FinalYaml.yaml', 'w') as f:

    f.close()

with open('/content/MergedDataset/FinalYaml.yaml', 'r') as f:
    data = yaml.load(f) 

## 학습 데이터, 검증 데이터 경로 지정
dic = {
'train' : f'{YourTrainPathFile}',
'val' : f'{YourValidPathFile}',

## 클래스 수
'nc' : 14 ,

## 종류
'names' : ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'bus', 'Zebra_Cross','G_Signal','R_Signal']
}

with open('/content/MergedDataset/FinalYaml.yaml', 'w') as f :
    yaml.dump(dic, f)


Zip 묶기 (shutil에 좋은 함수가 있네요)

In [ ]:
shutil.make_archive('MergedDataset', 'zip', '/content/MergedDataset')

옮겨주기

In [ ]:
shutil.move('/content/MergedDataset.zip','/content/gdrive/MyDrive/data')